In [3]:
# Torch.autograd를 사용한 자동 미분
# 매개변수는 주어진 매개변수에 대한 손실 함수의 gradient에 대해 조정 
# 이러한 변화도를 계산하려면 -> torch.autogrand 자동 미분 엔진 내장
# -> 모든 계산 그래프에 대한 변화도의 자동 계산을 지원
# y = Wx + b , x:input, parameters w and b...

import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
loss

tensor(1.0867, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [6]:
# 이 신경망에서, w와 b는 최적화를 해야 하는 매개변수이다.
# 이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야 한다.
# 이를 위해서 해당 텐서에 requires_grad 속성을 설정합니다.

# 연산 그래프를 구성하기 위해 텐서에 적용하는 함수는 사실 Function class의 객체이다.
# 이 객체는 순전파 방향으로 함수를 계산하는 방법과, 역방향 전파 단계에서 도함수를 계산하는 방법이 있다.

print(f"Gradient function for z = {z.grad_fn}")
print(f"Gradient function for loss = {loss.grad_fn}")

Gradient function for z = <AddBackward0 object at 0x7fa298115f70>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fa298115910>


In [7]:
# Gradient 계산하기
# 신경망에서 매개변수의 가중치를 최적화려면 매개변수에 대한 
# 손실함수의 도함수를 계산해야 한다.
# 즉, x와 y의 일부 고정값에서 (dloss/dw), (dloss/db)가 필요.
# 이러한 도함수를 계산하기 위해, loss backward()를 호출한 다음 w.grad와 b.grad에서 값을 가져온다. 

# No?
print(w.grad)
print(b.grad)

None
None


In [8]:
# loss.backword()
loss.backward()
print(w.grad)
print(b.grad)

# 연산 그래프의 잎(leaf) 노드들 중 requires_grad 속성이 True로 설정된 노드들의 grad 속성만 구할 수 있습니다. 
# 그래프의 다른 모든 노드에서는 변화도가 유효하지 않습니다.
# 성능 상의 이유로, 주어진 그래프에서의 backward를 사용한 변화도 계산은 한 번만 수행할 수 있습니다. 
# 만약 동일한 그래프에서 여러번의 backward 호출이 필요하면, 
# backward 호출 시에 retrain_graph=True를 전달해야 합니다.

tensor([[0.2609, 0.2737, 0.0042],
        [0.2609, 0.2737, 0.0042],
        [0.2609, 0.2737, 0.0042],
        [0.2609, 0.2737, 0.0042],
        [0.2609, 0.2737, 0.0042]])
tensor([0.2609, 0.2737, 0.0042])


In [9]:
# 변화도 추적 멈추기 
# requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원
# 하지만 순전파 연산의 경우 이러한 추적이 필요 없을 수 있는데, 이럴때는 아래와 같이
# 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있다.

z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [10]:
# 위와 같은 로직
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)
# 왜 굳이 변화도 추적을 멈춰야 하는 것일가 ?
# 1. 신경망의 일부 매개변수를 고정된 매개변수로 표시한다.
# 2. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상

False


In [11]:
# 연산 그래프에 대한 추가 정보

# When 순전파 -> autograd는 다음 두 가지 작업을 동시에 수행
# 1. 요청된 연산을 수행하여 결과 텐서를 계산
# 2. DAG(Directed Acyclic Graph)에 연산의 변화도 기능(gradient function)을 유지한다.

# When 역전파 -> DAG 뿌리(root)에서 .backward()가 호출될 때 시작된다. autograd는 이 때:
# 1. 각 .grad_fn 으로부터 변화도를 계산하고
# 2. 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate),
# 3. 연쇄 법칙을 사용하여, 모든 잎(leaf) 텐서들까지 propagate한다.